<a href="https://colab.research.google.com/github/Teniola17/Teniola17/blob/main/KL1_Estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **LIBRARY AND FUNCTIONS**

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from sklearn import linear_model
from sklearn import datasets
from sklearn.model_selection import train_test_split
from google.colab import files
import random
from sklearn.model_selection import KFold
import matplotlib.style as style
style.use('seaborn-colorblind')

%matplotlib inline
plt.style.use('seaborn-white')

def soft_threshold(rho,lamda):
    '''Soft threshold function used for normalized data and lasso regression'''
    if rho < - lamda:
        return (rho + lamda)
    elif rho >  lamda:
        return (rho - lamda)
    else:
        return 0

#tolerance
def tolerance(init_res, fin_res):
    diff = fin_res - init_res
    est_tol = diff.T @ diff
    return(est_tol.reshape(-1)[0])

def coordinate_descent_lasso(X,y,lamda = .01, num_iters=100, intercept = False, tol = 1e-10):
    '''Coordinate gradient descent for lasso regression - for normalized data.
    The intercept parameter allows to specify whether or not we regularize theta_0'''

    #Initialisation of useful values
    m,n = X.shape

    old_theta = np.linalg.inv(X.T @ X + np.identity(n)*lamda) @ (X.T @ y)

    #Looping until max number of iterations
    for i in range(num_iters):
        theta = old_theta
        #Looping through each coordinate
        for j in range(n):
            #Vectorized implementation
            X_j = X[:,j].reshape(-1,1)
            y_pred = X @ theta
            rho = X_j.T @ (y - y_pred  + theta[j]*X_j)
            c = sum(X[:,j]**2)

            #Checking intercept parameter
            if c == 0:
                theta[j] =  0
            else:
              if intercept == True:
                  if j == 0:
                      theta[j] =  rho
                  else:
                      theta[j] =  soft_threshold(rho/c, lamda/c)

              if intercept == False:
                  theta[j] =  soft_threshold(rho/c, lamda/c)

        est_tol = tolerance(old_theta, theta)
        if est_tol < tol:
          break
        else:
          old_theta = theta
    return theta.flatten()

<ipython-input-1-17a6d506fd40>:13: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  style.use('seaborn-colorblind')
<ipython-input-1-17a6d506fd40>:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [2]:
def coordinate_descent_liulasso(X,y,lamda = .01, d=0.3, num_iters=100, intercept = False, tol = 1e-10):
    '''Coordinate gradient descent for lasso regression - for normalized data.
    The intercept parameter allows to specify whether or not we regularize theta_0'''

    #Initialisation of useful values
    m,n = X.shape

    old_theta = np.linalg.inv(X.T @ X + np.identity(n)*lamda) @ (X.T @ y)

    #Looping until max number of iterations
    for i in range(num_iters):
        theta = old_theta
        #Looping through each                               coordinate
        for j in range(n):

            #Vectorized implementation
            X_j = X[:,j].reshape(-1,1)
            y_pred = X @ theta
            rho = X_j.T @ (y - y_pred  + theta[j]*X_j)+d*theta[j]

            #Checking intercept parameter
            if intercept == True:
                if j == 0:
                    theta[j] =  rho
                else:
                    theta[j] =  soft_threshold(rho/(1+sum(X[:, j]**2)), lamda/(1+sum(X[:, j]**2)))

            if intercept == False:
                theta[j] =  soft_threshold(rho/(1+sum(X[:, j]**2)), lamda/(1+sum(X[:, j]**2)))

        est_tol = tolerance(old_theta, theta)
        if est_tol < tol:
          break
        else:
          old_theta = theta

    return theta.flatten()


In [3]:
def coordinate_descent_goest(X, y, lamda = .01, alpha = .5, num_iters=100, intercept = False, tol = 1e-10):
    '''Coordinate gradient descent for lasso regression - for normalized data.
    The intercept parameter allows to specify whether or not we regularize theta_0'''

    #Initialisation of useful values
    m,n = X.shape

    old_theta = np.linalg.inv(X.T @ X + np.identity(n)*lamda) @ (X.T @ y)

    #Looping until max number of iterations
    for i in range(num_iters):
        theta = old_theta
        #Looping through each                               coordinate
        for j in range(n):

            #Vectorized implementation
            X_j = X[:,j].reshape(-1,1)
            y_pred = X @ theta
            rho = X_j.T @ (y - y_pred  + theta[j]*X_j)+lamda*(1-alpha)*theta[j]

            #Checking intercept parameter
            if intercept == True:
                if j == 0:
                    theta[j] =  rho
                else:
                    theta[j] =  soft_threshold(rho/((lamda*(1-alpha))+sum(X[:, j]**2)), (lamda*alpha)/((lamda*(1-alpha))+sum(X[:, j]**2)))

            if intercept == False:
                theta[j] =  soft_threshold(rho/((lamda*(1-alpha))+sum(X[:, j]**2)), (lamda*alpha)/((lamda*(1-alpha))+sum(X[:, j]**2)))

        est_tol = tolerance(old_theta, theta)
        if est_tol < tol:
          break
        else:
          old_theta = theta

    return theta.flatten()


In [4]:
def coordinate_descent_enet(X,y,lamda = .01, alpha = 0.5, num_iters=100, intercept = False, tol = 1e-10):
    '''Coordinate gradient descent for lasso regression - for normalized data.
    The intercept parameter allows to specify whether or not we regularize theta_0'''

    #Initialisation of useful values
    m,n = X.shape

    old_theta = np.linalg.inv(X.T @ X + np.identity(n)*lamda) @ (X.T @ y)

    #Looping until max number of iterations
    for i in range(num_iters):
        theta = old_theta
        #Looping through each                               coordinate
        for j in range(n):

            #Vectorized implementation
            X_j = X[:,j].reshape(-1,1)
            y_pred = X @ theta
            rho = X_j.T @ (y - y_pred  + theta[j]*X_j)+lamda*alpha

            #Checking intercept parameter
            if intercept == True:
                if j == 0:
                    theta[j] =  rho
                else:
                    theta[j] =  soft_threshold(rho/(sum(X[:, j]**2)+lamda*(1-alpha)), lamda/(sum(X[:, j]**2)+lamda*(1-alpha)))

            if intercept == False:
                theta[j] =  soft_threshold(rho/(sum(X[:, j]**2)+lamda*(1-alpha)), lamda/(sum(X[:, j]**2)+lamda*(1-alpha)))

        est_tol = tolerance(old_theta, theta)
        if est_tol < tol:
          break
        else:
          old_theta = theta

    return theta.flatten()

In [5]:
def coordinate_descent_KL1(X, y, lamda = .01, num_iters=100, intercept = False, tol = 1e-10):
    '''Coordinate gradient descent for lasso regression - for normalized data.
    The intercept parameter allows to specify whether or not we regularize theta_0'''

    #Initialisation of useful values
    m,n = X.shape

    old_theta = np.linalg.inv(X.T @ X + np.identity(n)*lamda) @ (X.T @ y)

    #Looping until max number of iterations
    for i in range(num_iters):
        theta = old_theta
        #Looping through each                               coordinate
        for j in range(n):

            #Vectorized implementation
            X_j = X[:,j].reshape(-1,1)
            y_pred = X @ theta
            rho = X_j.T @ (y - y_pred  + theta[j]*X_j)-theta[j]

            #Checking intercept parameter
            if intercept == True:
                if j == 0:
                    theta[j] =  rho
                else:
                    theta[j] =  soft_threshold(rho/(1+sum(X[:, j]**2)), lamda/(1+sum(X[:, j]**2)))

            if intercept == False:
                theta[j] =  soft_threshold(rho/(1+sum(X[:, j]**2)), lamda/(1+sum(X[:, j]**2)))

        est_tol = tolerance(old_theta, theta)
        if est_tol < tol:
          break
        else:
          old_theta = theta

    return theta.flatten()

In [6]:
#OLS, Liu, Ridge and KL Estimators
def ridge_estimator(X,y,lamda = .01):
  m,n = X.shape
  theta = np.linalg.inv(X.T @ X + np.identity(n)*lamda) @ (X.T @ y)
  return theta.flatten()

def ols_estimator(X,y):
  theta = np.linalg.inv(X.T @ X) @ (X.T @ y)
  return theta.flatten()

def liu_estimator(X,y,d):
  m,n = X.shape
  old_theta = np.linalg.inv(X.T @ X) @ (X.T @ y)
  theta = np.linalg.inv(X.T @ X + np.identity(n)) @ (X.T @ X + np.identity(n)*d) @ old_theta
  return theta.flatten()

def mliu_estimator(X,y,d):
  m,n = X.shape
  old_theta = np.linalg.inv(X.T @ X) @ (X.T @ y)
  theta = np.linalg.inv(X.T @ X + np.identity(n)) @ (X.T @ X - np.identity(n)*d) @ old_theta
  return theta.flatten()

def kl_estimator(X,y,lamda):
  m,n = X.shape
  old_theta = np.linalg.inv(X.T @ X) @ (X.T @ y)
  theta = np.linalg.inv(X.T @ X + np.identity(n)*lamda) @ (X.T @ X - np.identity(n)*lamda) @ old_theta
  return theta.flatten()

In [7]:
#Function to compute mean square error
def mse_beta(est_beta, real_beta):
    diff = est_beta - real_beta
    mse_b = diff.T @ diff
    return(mse_b.reshape(-1)[0])

In [8]:
#Confusion matrix function for Beta Coefficients
def cm_par(est_beta, real_beta):
  actual = [1 if i!=0 else 0 for i in real_beta]
  predicted = [1 if i!=0 else 0 for i in est_beta]
  data = {'beta_predict':predicted, 'beta_actual':actual}
  cm_df = pd.DataFrame(data, columns=['beta_actual','beta_predict'])
  cm_res = pd.crosstab(cm_df['beta_actual'], cm_df['beta_predict'], rownames=['Actual'], colnames=['Predicted'])#, margins = True)
  cm_res_un = cm_res.unstack().reorder_levels(('Actual','Predicted'))
  try:
    fnr = cm_res_un.loc[1,0]
  except:
    fnr = np.nan
  try:
    fpr = cm_res_un.loc[0,1]
  except:
    fpr = np.nan
  return(fnr, fpr)

**SIMULATION: Validation, Training and Test**

In [9]:
#First Simulation
def gen_dat(bet_bb, pp = 10, nn = 30, rho = 0.5, sig=5):
  cov = np.zeros((pp,pp))
  for i in range(pp):
    for j in range(pp):
      cov[i,j] = rho**(abs(i - j))
  mean = np.zeros(pp)
  x_mat = np.random.multivariate_normal(mean, cov, nn)
  df_x = pd.DataFrame(x_mat)
  eps = np.random.normal(loc = 0.0, scale = sig, size = nn)
  y_vec = (x_mat @ bet_bb) + eps
  df_x.columns = ['x'+str(i+1) for i in range(pp)]
  df_x.insert(0,'y',y_vec)
  return df_x

In [ ]:
#SIMULATION PART - High Dimension
#beta_int = np.array([0.5]*15 + [0]*25 + [0.7]*15 + [0]*30 + [0.7]*15) #sim A - 55 zeros
#beta_int = np.array([0.5]*15 + [0]*25 + [0]*15 + [0]*30 + [0]*15) #sim B - 85 zeros
#beta_int = np.array([0.5]*15 + [0.2]*25 + [0.7]*15 + [0.4]*30 + [0.9]*15) #sim C
#sim_p = 100; sim_times = 50; sim_size = [200]

In [10]:
#SIMULATION PART - Low Dimension
#beta_int = np.array([3,1.5,0,0,2,0,0,0])  #sim A
#beta_int = np.array([3,0,0,0,0,0,0,0])#sim B
beta_int = np.array([3,1.5,2,1.6,2,1.5,1.5,1.5]) #sim C
sim_p = 8; sim_times = 50; sim_size = [400]#100, 200, 400]


In [11]:
whole_tab_sim_df = pd.DataFrame(columns=['n','p','rho','sig','stats','lasso', 'liulasso', 'goest', 'enet', 'kl1', 'ridge'])
sim_tab_row = 1

for sim_sig in [5,10]:
  for sim_rho in [0.99]:#0.5,0.8,0.99]:
    for sim_n in sim_size:
      test_mse_sim_df = pd.DataFrame(columns=['lasso', 'liulasso', 'goest', 'enet', 'kl1', 'ridge'])
      beta_mse_sim_df = pd.DataFrame(columns=['lasso', 'liulasso', 'goest', 'enet', 'kl1', 'ridge'])
      fpr_sim_df = pd.DataFrame(columns=['lasso', 'liulasso', 'goest', 'enet', 'kl1', 'ridge'])
      fnr_sim_df = pd.DataFrame(columns=['lasso', 'liulasso', 'goest', 'enet', 'kl1', 'ridge'])

      test_row = 1
      for sim_no in range(sim_times):
        dat = gen_dat(beta_int, pp = sim_p, nn = sim_n, sig = sim_sig)
        dat = dat.to_numpy()
        yr = dat[:, 1].reshape(-1, 1)
        Xr = np.delete(dat, -1, axis=1)
        yr = yr.reshape(yr.shape[0], 1)

        xx_trainval, xx_test, yy_trainval, yy_test = train_test_split(Xr, yr, test_size = 0.20, random_state = 15)

        xx_train, xx_valid, yy_train, yy_valid = train_test_split(xx_trainval, yy_trainval, test_size = 0.25, random_state = 15)

        #Empty dataframe to hold mean and median for each parameter combination
        fold_mses_df = pd.DataFrame(columns=['lambda','alpha','d','lasso','liulasso','goest','enet','kl1','ridge'])

        row_num = 1
        for lam in [0.1*(i+1) for i in range(10)]: #[0.1*(i+1) for i in range(9)]:
          for a_p in [0.1*(i+1) for i in range(10)]:
            for d_p in [0.1*(i+1) for i in range(10)]:

              lasso_mod = coordinate_descent_lasso(xx_train, yy_train, lamda = lam, num_iters=100, intercept = False)
              liulasso_mod = coordinate_descent_liulasso(xx_train, yy_train, lamda = lam, d = d_p, num_iters=100, intercept = False)
              goest_mod = coordinate_descent_goest(xx_train, yy_train, lamda = lam, alpha = a_p, num_iters=100, intercept = False)
              enet_mod = coordinate_descent_enet(xx_train, yy_train, lamda = lam, alpha = a_p, num_iters=100, intercept = False)
              kl1_mod = coordinate_descent_KL1(xx_train, yy_train, lamda = lam, num_iters=100, intercept = False)
              ridge_mod = ridge_estimator(xx_train, yy_train, lamda = lam)

              msel_lasso = np.square(np.subtract(yy_valid,xx_valid@lasso_mod)).mean()
              msel_liulasso = np.square(np.subtract(yy_valid,xx_valid@liulasso_mod)).mean()
              msel_goest = np.square(np.subtract(yy_valid,xx_valid@goest_mod)).mean()
              msel_enet = np.square(np.subtract(yy_valid,xx_valid@enet_mod)).mean()
              msel_kl1 = np.square(np.subtract(yy_valid,xx_valid@kl1_mod)).mean()
              msel_ridge = np.square(np.subtract(yy_valid,xx_valid@ridge_mod)).mean()

              fold_mses_df.loc[row_num] = [lam, a_p, d_p, msel_lasso, msel_liulasso, msel_goest, msel_enet, msel_kl1, msel_ridge]

              row_num+=1

        all_estimators_names = ['lasso','liulasso','goest','enet','klenet','kl1','kl2','goestd','kl3','kl4','ridge']

        lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[0]].idxmin()].to_list()[0:3]
        lasso_modt = coordinate_descent_lasso(xx_trainval, yy_trainval, lamda = lam, num_iters=100, intercept = False)

        lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[1]].idxmin()].to_list()[0:3]
        liulasso_modt = coordinate_descent_liulasso(xx_trainval, yy_trainval, lamda = lam, d = d_p, num_iters=100, intercept = False)

        lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[2]].idxmin()].to_list()[0:3]
        goest_modt = coordinate_descent_goest(xx_trainval, yy_trainval, lamda = lam, alpha = a_p, num_iters=100, intercept = False)

        lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[3]].idxmin()].to_list()[0:3]
        enet_modt = coordinate_descent_enet(xx_trainval, yy_trainval, lamda = lam, alpha = a_p, num_iters=100, intercept = False)

        lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[5]].idxmin()].to_list()[0:3]
        kl1_modt = coordinate_descent_KL1(xx_trainval, yy_trainval, lamda = lam, num_iters=100, intercept = False)

        lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[10]].idxmin()].to_list()[0:3]
        ridge_modt = ridge_estimator(xx_trainval, yy_trainval, lamda = lam)

        #Estimate MSE BETA
        mseb_lasso = mse_beta(lasso_modt, beta_int)
        mseb_liulasso = mse_beta(liulasso_modt, beta_int)
        mseb_goest = mse_beta(goest_modt, beta_int)
        mseb_enet = mse_beta(enet_modt, beta_int)
        mseb_kl1 = mse_beta(kl1_modt, beta_int)
        mseb_ridge = mse_beta(ridge_modt, beta_int)
        beta_mse_sim_df.loc[test_row] = [mseb_lasso, mseb_liulasso, mseb_goest, mseb_enet, mseb_kl1, mseb_ridge]

        #False Negative and False Positve
        fnr_lasso, fpr_lasso = cm_par(lasso_modt, beta_int)
        fnr_liulasso, fpr_liulasso = cm_par(liulasso_modt, beta_int)
        fnr_goest, fpr_goest = cm_par(goest_modt, beta_int)
        fnr_enet, fpr_enet = cm_par(enet_modt, beta_int)
        fnr_kl1, fpr_kl1 = cm_par(kl1_modt, beta_int)
        fnr_ridge, fpr_ridge = cm_par(ridge_modt, beta_int) #(np.nan, np.nan)
        fnr_sim_df.loc[test_row] = [fnr_lasso, fnr_liulasso, fnr_goest, fnr_enet, fnr_kl1, fnr_ridge]
        fpr_sim_df.loc[test_row] = [fpr_lasso, fpr_liulasso, fpr_goest, fpr_enet, fpr_kl1, fpr_ridge]

        #Estimate MSE on Y
        mset_lasso = np.square(np.subtract(yy_test,xx_test@lasso_modt)).mean()
        mset_liulasso = np.square(np.subtract(yy_test,xx_test@liulasso_modt)).mean()
        mset_goest = np.square(np.subtract(yy_test,xx_test@goest_modt)).mean()
        mset_enet = np.square(np.subtract(yy_test,xx_test@enet_modt)).mean()
        mset_kl1 = np.square(np.subtract(yy_test,xx_test@kl1_modt)).mean()
        mset_ridge = np.square(np.subtract(yy_test,xx_test@ridge_modt)).mean()
        test_mse_sim_df.loc[test_row] = [mset_lasso, mset_liulasso, mset_goest, mset_enet, mset_kl1, mset_ridge]
        test_row+=1

      fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
      beta_mse_sim_df.plot(kind='box', fontsize=14, ax=ax1)#, color=dict(boxes='blue', whiskers='blue', medians='green', caps='blue'))
      ax1.set_ylabel(r'$\beta$'+'-MSE',fontdict={'fontsize':15}), ax1.tick_params(axis='x', labelrotation=45)
      test_mse_sim_df.plot(kind='box', fontsize=14, ax=ax2)#, color=dict(boxes='blue', whiskers='blue', medians='green', caps='blue')) grid=True
      ax2.set_ylabel('TMSE',fontdict={'fontsize':15}), ax2.tick_params(axis='x', labelrotation=45)
      plt.savefig(str(sim_n)+"_"+str(sim_p)+"_"+str(sim_rho)+"_"+str(sim_sig)+".png")
      files.download(str(sim_n)+"_"+str(sim_p)+"_"+str(sim_rho)+"_"+str(sim_sig)+".png")

      whole_tab_sim_df.loc[sim_tab_row] = [sim_n,sim_p,sim_rho,sim_sig,"Beta_med"] + beta_mse_sim_df.median().to_list()
      whole_tab_sim_df.loc[sim_tab_row+1] = [sim_n,sim_p,sim_rho,sim_sig,"Beta_mean"] + beta_mse_sim_df.mean().to_list()
      whole_tab_sim_df.loc[sim_tab_row+2] = [sim_n,sim_p,sim_rho,sim_sig,"Beta_sem"] + beta_mse_sim_df.sem().to_list()

      whole_tab_sim_df.loc[sim_tab_row+3] = [sim_n,sim_p,sim_rho,sim_sig,"Test_med"] + test_mse_sim_df.median().to_list()
      whole_tab_sim_df.loc[sim_tab_row+4] = [sim_n,sim_p,sim_rho,sim_sig,"Test_mean"] + test_mse_sim_df.mean().to_list()
      whole_tab_sim_df.loc[sim_tab_row+5] = [sim_n,sim_p,sim_rho,sim_sig,"Test_sem"] + test_mse_sim_df.sem().to_list()

      whole_tab_sim_df.loc[sim_tab_row+6] = [sim_n,sim_p,sim_rho,sim_sig,"fnr"] + fnr_sim_df.median().to_list()
      whole_tab_sim_df.loc[sim_tab_row+7] = [sim_n,sim_p,sim_rho,sim_sig,"fpr"] + fpr_sim_df.median().to_list()
      whole_tab_sim_df.loc[sim_tab_row+8] = ['n','p','rho','sig','stat','lasso', 'liulasso', 'goest', 'enet', 'kl1', 'ridge']

      sim_tab_row+=9


KeyboardInterrupt: ignored

In [12]:
pd.DataFrame(whole_tab_sim_df).to_csv('whole_tab_sim_dff.csv', encoding = 'utf-8-sig')
files.download('whole_tab_sim_dff.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
whole_tab_sim_df

,n,p,rho,sig,stats,lasso,liulasso,goest,enet,kl1,ridge


**PRACTICAL APPLICATION**

In [23]:
#datasuper=files.upload()
#dat=pd.read_csv("flu.csv")
#datasuper=files.upload()
#dat=pd.read_csv("superconduct.csv")
#dataasphalt=files.upload()
#dat=pd.read_csv("data asphalt.csv")
#prostatedata=files.upload()
#dat=pd.read_csv("Prostate data.csv")
ahdd=files.upload()
dat=pd.read_csv("anti.csv")

Saving anti.csv to anti.csv


**K-Fold Classification Approach**

In [24]:
# Format Data
dat = dat.to_numpy()
yr = dat[:, 1].reshape(-1, 1)
Xr = np.delete(dat, -1, axis=1)

yr = yr.reshape(yr.shape[0], 1)

X, xx_test, y, yy_test = train_test_split(Xr, yr, test_size = 0.20, random_state = 15)


In [ ]:
random.seed(100)
import time

#Empty dataframe to hold mean and median for each parameter combination
fold_mses_df = pd.DataFrame(columns=['lambda','alpha','d','lasso','liulasso','goest','enet','kl1','ridge'])
fold_med_df = pd.DataFrame(columns=['lambda','alpha','d','lasso','liulasso','goest','enet','kl1','ridge'])
fold_times_df = pd.DataFrame(columns=['lambda','alpha','d','lasso','liulasso','goest','enet','kl1','ridge'])
fold_medtime_df = pd.DataFrame(columns=['lambda','alpha','d','lasso','liulasso','goest','enet','kl1','ridge'])
row_num = 1
for lam in [0.1*(i+1) for i in range(10)]: #[0.1*(i+1) for i in range(9)]:
  for a_p in [0.1*(i+1) for i in range(10)]:
    for d_p in [0.1*(i+1) for i in range(10)]:
      #Initiate empty list to hold mses for folds
      msel_lasso, msel_liulasso, msel_goest, msel_enet = np.array([]), np.array([]), np.array([]), np.array([])
      time_lasso, time_liulasso, time_goest, time_enet = np.array([]), np.array([]), np.array([]), np.array([])
      msel_kl1, msel_ridge = np.array([]), np.array([])
      time_kl1, time_ridge = np.array([]), np.array([])
      kf3 = KFold(n_splits=10, shuffle=False)
      sp = kf3.split(y)
      for train_index, test_index in sp:
        X_train=X[train_index,:]
        y_train=y[train_index,:]
        X_test=X[test_index,:]
        y_test=y[test_index,:]
        start1= time.time()
        lasso_mod = coordinate_descent_lasso(X_train, y_train,lamda = lam, num_iters=100, intercept = False)
        stop1= time.time()
        start2= time.time()
        liulasso_mod = coordinate_descent_liulasso(X_train, y_train, lamda = lam, d = d_p, num_iters=100, intercept = False)
        stop2= time.time()
        start3= time.time()
        goest_mod = coordinate_descent_goest(X_train, y_train, lamda = lam, alpha = a_p, num_iters=100, intercept = False)
        stop3= time.time()
        start4= time.time()
        enet_mod = coordinate_descent_enet(X_train, y_train, lamda = lam, alpha = a_p, num_iters=100, intercept = False)
        stop4= time.time()
        start5= time.time()
        kl1_mod = coordinate_descent_KL1(X_train, y_train, lamda = lam, num_iters=100, intercept = False)
        stop5= time.time()
        start6= time.time()
        ridge_mod = ridge_estimator(X_train, y_train, lamda = lam)
        stop6= time.time()
        msel_lasso = np.append(msel_lasso,np.square(np.subtract(y_test,X_test@lasso_mod)).mean())
        msel_liulasso = np.append(msel_liulasso,np.square(np.subtract(y_test,X_test@liulasso_mod)).mean())
        msel_goest = np.append(msel_goest,np.square(np.subtract(y_test,X_test@goest_mod)).mean())
        msel_enet = np.append(msel_enet,np.square(np.subtract(y_test,X_test@enet_mod)).mean())
        msel_kl1 = np.append(msel_kl1,np.square(np.subtract(y_test,X_test@kl1_mod)).mean())
        msel_ridge = np.append(msel_ridge,np.square(np.subtract(y_test,X_test@ridge_mod)).mean())

        time_lasso = np.append(time_lasso,np.subtract(stop1,start1).mean())
        time_liulasso = np.append(time_liulasso,np.subtract(stop2,start2).mean())
        time_goest = np.append(time_goest,np.subtract(stop3,start3).mean())
        time_enet = np.append(time_enet,np.subtract(stop4,start4).mean())
        time_kl1 =np.append(time_kl1,np.subtract(stop5,start5).mean())
        time_ridge = np.append(time_ridge,np.subtract(stop6,start6).mean())

      msel_lasso_mean, msel_liulasso_mean, msel_goest_mean, msel_enet_mean = np.mean(msel_lasso), np.mean(msel_liulasso), np.mean(msel_goest), np.mean(msel_enet)
      msel_kl1_mean, msel_ridge_mean = np.mean(msel_kl1), np.mean(msel_ridge)
      fold_mses_df.loc[row_num] = [lam,a_p,d_p,msel_lasso_mean, msel_liulasso_mean, msel_goest_mean, msel_enet_mean, msel_kl1_mean, msel_ridge_mean]

      time_lasso_mean, time_liulasso_mean, time_goest_mean, time_enet_mean = np.mean(time_lasso), np.mean(time_liulasso), np.mean(time_goest), np.mean(time_enet)
      time_kl1_mean, time_ridge_mean = np.mean(time_kl1), np.mean(time_ridge)
      fold_times_df.loc[row_num] = [lam,a_p,d_p,time_lasso_mean, time_liulasso_mean, time_goest_mean, time_enet_mean, time_kl1_mean, time_ridge_mean]

      msel_lasso_med, msel_liulasso_med, msel_goest_med, msel_enet_med = np.median(msel_lasso), np.median(msel_liulasso), np.median(msel_goest), np.median(msel_enet)
      msel_kl1_med, msel_ridge_med = np.median(msel_kl1), np.median(msel_ridge)
      fold_med_df.loc[row_num] = [lam,a_p,d_p,msel_lasso_med, msel_liulasso_med, msel_goest_med, msel_enet_med, msel_kl1_med, msel_ridge_med ]

      time_lasso_med, time_liulasso_med, time_goest_med, time_enet_med = np.median(time_lasso), np.median(time_liulasso), np.median(time_goest), np.median(time_enet)
      time_kl1_med, time_ridge_med = np.median(time_kl1), np.median(time_ridge)
      fold_medtime_df.loc[row_num] = [lam,a_p,d_p,time_lasso_med, time_liulasso_med, time_goest_med, time_enet_med, time_kl1_med, time_ridge_med ]

      row_num+=1


In [17]:
pd.DataFrame(fold_mses_df).to_csv('fold_mses_dff.csv', encoding = 'utf-8-sig')
files.download('fold_mses_dff.csv')

pd.DataFrame(fold_med_df).to_csv('fold_med_dff.csv', encoding = 'utf-8-sig')
files.download('fold_med_dff.csv')

pd.DataFrame(fold_times_df).to_csv('fold_times_dff.csv', encoding = 'utf-8-sig')
files.download('fold_times_dff.csv')

pd.DataFrame(fold_medtime_df).to_csv('fold_medtime_dff.csv', encoding = 'utf-8-sig')
files.download('fold_metimed_dff.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
fold_mses_dff = files.upload()
fold_mses_df = pd.read_csv("fold_mses_dff.csv")

fold_med_dff = files.upload()
fold_med_df = pd.read_csv("fold_med_dff.csv")

In [18]:
fold_mses_df = fold_mses_df[['lambda','alpha','d','lasso','liulasso','goest','enet','kl1','ridge']]
fold_med_df = fold_med_df[['lambda','alpha','d','lasso','liulasso','goest','enet','kl1','ridge']]

In [21]:
all_estimators_names = ['lasso','liulasso','goest','enet','kl1','ridge']
hyper_par_mse_df = pd.DataFrame(columns=['estimator','lambda','alpha','d','mse'])
row_no = 1
for i in range(len(all_estimators_names)):
 ggg = fold_mses_df.loc[fold_mses_df[all_estimators_names[i]].idxmin()].to_list()
 hyper_par_mse_df.loc[row_no] = [all_estimators_names[i]] + ggg[0:3] + [ggg[i+3]]
 row_no+=1

TypeError: ignored

In [ ]:
pd.DataFrame(hyper_par_mse_df).to_csv('hyper_par_mse_dff.csv', encoding = 'utf-8-sig')
files.download('hyper_par_mse_dff.csv')

In [ ]:
hyper_par_mse_dff = files.upload()
hyper_par_mse_df = pd.read_csv("hyper_par_mse_dff.csv")

In [ ]:
hyper_par_mse_df[['estimator','lambda','alpha','d','mse']]

In [ ]:
xx_train, yy_train = X, y

lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[0]].idxmin()].to_list()[0:3]
lasso_modt = coordinate_descent_lasso(xx_train, yy_train,lamda = lam, num_iters=100, intercept = False)

lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[1]].idxmin()].to_list()[0:3]
liulasso_modt = coordinate_descent_liulasso(xx_train, yy_train, lamda = lam, d = d_p, num_iters=100, intercept = False)

lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[2]].idxmin()].to_list()[0:3]
goest_modt = coordinate_descent_goest(xx_train, yy_train, lamda = lam, alpha = a_p, num_iters=100, intercept = False)

lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[3]].idxmin()].to_list()[0:3]
enet_modt = coordinate_descent_enet(xx_train, yy_train, lamda = lam, alpha = a_p, num_iters=100, intercept = False)

lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[5]].idxmin()].to_list()[0:3]
kl1_modt = coordinate_descent_KL1(xx_train, yy_train, lamda = lam, num_iters=100, intercept = False)

lam, d_p, a_p = fold_mses_df.loc[fold_mses_df[all_estimators_names[10]].idxmin()].to_list()[0:3]
ridge_modt = ridge_estimator(xx_train, yy_train, lamda = lam)

mset_lasso = np.square(np.subtract(yy_test,xx_test@lasso_modt)).mean()
mset_liulasso = np.square(np.subtract(yy_test,xx_test@liulasso_modt)).mean()
mset_goest = np.square(np.subtract(yy_test,xx_test@goest_modt)).mean()
mset_enet = np.square(np.subtract(yy_test,xx_test@enet_modt)).mean()
mset_kl1 = np.square(np.subtract(yy_test,xx_test@kl1_modt)).mean()
mset_ridge = np.square(np.subtract(yy_test,xx_test@ridge_modt)).mean()

coeff_beta_modt = [lasso_modt,liulasso_modt,goest_modt,enet_modt,kl1_modt,ridge_modt]
mse_test_beta_modt = [mset_lasso, mset_liulasso, mset_goest, mset_enet, mset_kl1, mset_ridge]

In [ ]:
#xx_test.shape[1]
coeff_beta_modt_df = pd.DataFrame(coeff_beta_modt).transpose()
var_name = ['x'+str(i) for i in range(xx_train.shape[1])]
coeff_beta_modt_df.insert(loc=0, column='Variable', value=var_name)
next_row = max(list(coeff_beta_modt_df.index.values)) + 1
coeff_beta_modt_df.loc[next_row] = ['mses'] + mse_test_beta_modt
coeff_beta_modt_df.columns = ['variable','lasso','liulasso','goest','enet','kl1','ridge']

In [ ]:
pd.DataFrame(coeff_beta_modt_df).to_csv('coeff_beta_modt_dff.csv', encoding = 'utf-8-sig')
files.download('coeff_beta_modt_dff.csv')

In [ ]:
coeff_beta_modt_dff = files.upload()
coeff_beta_modt_df = pd.read_csv("coeff_beta_modt_dff.csv")

In [ ]:
coeff_beta_modt_df[['variable','lasso','liulasso','goest','enet','kl1','ridge']]

In [ ]:
#Empty dataframe to hold mean and median for each parameter combination
fold_mses_df = pd.DataFrame(columns=['lambda','alpha','d','lasso','liulasso','goest','enet','kl1','ridge'])
fold_med_df = pd.DataFrame(columns=['lambda','alpha','d','lasso','liulasso','goest','enet','kl1','ridge'])

row_num = 1
for lam in [0.1*(i+1) for i in range(10)]: #[0.1*(i+1) for i in range(9)]:
  for a_p in [0.1*(i+1) for i in range(10)]:
    for d_p in [0.1*(i+1) for i in range(10)]:
      lasso_mod = coordinate_descent_lasso(xx_train, yy_train,lamda = lam, num_iters=100, intercept = False)
      liulasso_mod = coordinate_descent_liulasso(xx_train, yy_train, lamda = lam, d = d_p, num_iters=100, intercept = False)
      goest_mod = coordinate_descent_goest(xx_train, yy_train, lamda = lam, alpha = a_p, num_iters=100, intercept = False)
      enet_mod = coordinate_descent_enet(xx_train, yy_train, lamda = lam, alpha = a_p, num_iters=100, intercept = False)
      kl1_mod = coordinate_descent_KL1(xx_train, yy_train, lamda = lam, num_iters=100, intercept = False)
      ridge_mod = ridge_estimator(xx_train, yy_train, lamda = lam)
      msel_lasso = np.append(msel_lasso,np.square(np.subtract(y_test,X_test@lasso_mod)).mean())
      msel_liulasso = np.append(msel_liulasso,np.square(np.subtract(y_test,X_test@liulasso_mod)).mean())
      msel_goest = np.append(msel_goest,np.square(np.subtract(y_test,X_test@goest_mod)).mean())
      msel_enet = np.append(msel_enet,np.square(np.subtract(y_test,X_test@enet_mod)).mean())
      msel_kl1 = np.append(msel_kl1,np.square(np.subtract(y_test,X_test@kl1_mod)).mean())
      msel_ridge = np.append(msel_ridge,np.square(np.subtract(y_test,X_test@ridge_mod)).mean())

      msel_lasso_mean, msel_liulasso_mean, msel_goest_mean, msel_enet_mean = np.mean(msel_lasso), np.mean(msel_liulasso), np.mean(msel_goest), np.mean(msel_enet)
      msel_kl1_mean, msel_ridge_mean = np.mean(msel_kl1), np.mean(msel_ridge)
      fold_mses_df.loc[row_num] = [lam,a_p,d_p,msel_lasso_mean, msel_liulasso_mean, msel_goest_mean, msel_enet_mean,msel_kl1_mean, msel_ridge_mean]

      msel_lasso_med, msel_liulasso_med, msel_goest_med, msel_enet_med = np.median(msel_lasso), np.median(msel_liulasso), np.median(msel_goest), np.median(msel_enet)
      msel_kl1_med, msel_ridge_med = np.median(msel_kl1), np.median(msel_ridge)
      fold_med_df.loc[row_num] = [lam,a_p,d_p,msel_lasso_med, msel_liulasso_med, msel_goest_med, msel_enet_med, msel_kl1_med, msel_ridge_med ]

      row_num+=1
